# Tutorial2: generating synthetic ngEHT observations

---
This tutorial demonstrates synthesis of ngEHT measurements of a single orbiting hot-spot. This tutorial assumes a sensor xr.Dataset was generated and saved and is ready to be loaded (see Tutorial1 for more details). The sensor dataset contains information about rays geometry assuming a black-hole with a given spin and mass. In this example we used spin=0 with r_g=2 to generate rays, which results in an inner most stable circular orbit (ISCO) of 3M [1]. 


[1] Overleaf notes: https://www.overleaf.com/project/60ff0ece5aa4f90d07f2a417

In [21]:
import bhnerf
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [3]:
"""
Generate an initial frame (3D DataArray) with single Gaussian hotspot
"""
spatial_res = (64, 64, 64) # (nx, ny, nz)

r_isco = 3.0 
orbit_radius = 3.5 
std = .4

rot_angle = 0.0  # initial orbital angle
rot_axis = [-0.5, 0.0, 0.8660254]
initial_frame = bhnerf.emission.generate_hotspot_xr(spatial_res, rot_axis, rot_angle, orbit_radius, std, r_isco)

In [4]:
"""
Progegate initial frame through time (4D DataArray) with a velocity field as a function of radius.
This results in a shearing hotspot.
"""
nt = 128
orbit_period = orbit_radius**(-3./2.) 
velocity_field = lambda r: (1.0 / orbit_period) * r**(-3/2)
emission = bhnerf.emission.generate_orbit(initial_frame, nt, velocity_field, rot_axis)

# Normalize emission values 
normalization_factor = 0.02
emissions = emission * normalization_factor

In [5]:
"""
Slider visualization to illustrate the volumetric shearing
This visualization requires ipyvolume: https://ipyvolume.readthedocs.io/en/latest/
"""
import ipyvolume as ipv
from ipywidgets import interact
import ipywidgets as widgets

extent = [(float(emission[dim].min()), float(emission[dim].max())) for dim in ('x', 'y', 'z')]
@interact(t=widgets.IntSlider(min=0, max=emission.t.size-1, step=1, value=0))
def plot_vol(t):
    ipv.figure()
    ipv.view(0, -60, distance=2.5)
    ipv.volshow(emission.isel(t=t), extent=extent, memorder='F', level=[0, 0.2, 0.7], opacity=[0, 0.2, 0.3], controls=False)
    ipv.show()

interactive(children=(IntSlider(value=0, description='t', max=127), Output()), _dom_classes=('widget-interact'…

In [20]:
"""
Load a precomuted sensor Dataset and integrate over the 4D emission field to get 3D image-plane movie.  
"""
sensor = xr.load_dataset('../sensors/a0.00_th1.57_ngeo100_npix4096.nc')

# Integrate emission within 5M of the black hole center
sensor = sensor.where(sensor.r < 5).fillna(0.0)
image_pixels = bhnerf.emission.integrate_rays(emission, sensor)
image_plane = image_pixels.data.reshape(nt, sensor.num_alpha, sensor.num_beta)
image_plane_xr = xr.DataArray(image_plane, dims=['t', 'alpha', 'beta'])

In [23]:
"""
Generate synthetic ngEHT observations of the image plane.
This requires eht-imaging: https://github.com/achael/eht-imaging
"""
import ehtim as eh

fov = 85.0               # field of view in uas 
obs_params = {
    'mjd': 57851,
    'array': eh.array.load_txt('../eht_arrays/ngEHT.txt'),
    'timetype': 'GMST',
    'nt': 128,           # number of time samples 
    'tstart': 2.0,       # start of observations
    'tstop': 2.0 + 2/3,  # end of observation 
    'tint': 15.0         # integration time
}
obs_empty = bhnerf.observation.empty_eht_obs(**obs_params)


obs_args = {
    'psize': fov / sensor.num_alpha * eh.ehc.RADPERUAS,
    'ra': obs_empty.ra, 
    'dec': obs_empty.dec,
    'rf': obs_empty.rf, 
    'mjd': obs_empty.mjd
}
times = np.linspace(obs_params['tstart'], obs_params['tstop'], obs_params['nt'])
movie = eh.movie.Movie(image_plane, times=times, **obs_args)
obs = movie.observe_same(obs_empty)

Producing clean visibilities from movie with nfft FT . . . 
Adding gain + phase errors to data and applying a priori calibration . . . 
Adding thermal noise to data . . . 


In [45]:
"""
Visualize image-plane movie and ngEHT measurements
"""
%matplotlib widget
fig, ax = plt.subplots(1, 3, figsize=(12,3.5))
ax[0].set_title('Image plane')
ax[1].set_title('UV coverage')
ax[2].set_title('Visibility amplitudes')
anim = image_plane_xr.visualization.animate(ax=ax[0], cmap='afmhot', add_colorbar=False)
bhnerf.observation.plot_uv_coverage(obs_empty, ax=ax[1], cmap_ticks=[0,0.2, 0.4, 0.6], fontsize=11)
obs.plotall('uvdist', 'amp', axis=ax[2])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …